# Bank Customer Churn Modeling 

Bank customer churn is a measure why customers are leaving a business. In this notebook we will be looking at customer churn in this bank company. we will buidl a deep learning model to predict the churn and use precision, recall, f1 score to measure performace of our model. we will handle imbalced datset using various techniques and imporove f1 score 

In [5]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')

## Load the data 

In [7]:
df = pd.read_csv("csvs/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [12]:
df.shape

(10000, 14)

### First of all, drop the customer id col as it is of no use

In [13]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [22]:
df.drop('CustomerId', axis='columns', inplace=True)

In [23]:
df.dtypes

RowNumber            int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [24]:
df.head()

,RowNumber,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
df.drop('Surname', axis='columns', inplace=True)
df.drop('Geography', axis='columns', inplace=True)

In [27]:
df.head()

,RowNumber,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,619,Female,42,2,0.00,1,1,1,101348.88,1
1,2,608,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,502,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,699,Female,39,1,0.00,2,0,0,93826.63,0
4,5,850,Female,43,2,125510.82,1,1,1,79084.10,0


In [28]:
df.dtypes

RowNumber            int64
CreditScore          int64
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

### Many columns have yes/no values. lets print unique values in object columns to see data values

In [29]:
def unique_col_value(df):
    for column in df:
        if df[column].dtypes=='object':
            print(f'{column}: {df[column].unique()}')

In [33]:
unique_col_value(df)

Gender: ['Female' 'Male']


Converting yes/no into 1 and 0

In [34]:
df['Gender'].replace({'Female':1, 'Male':0}, inplace=True)

In [35]:
df.Gender.unique()

array([1, 0], dtype=int64)

In [37]:
df.head(10)

,RowNumber,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,619,1,42,2,0.00,1,1,1,101348.88,1
1,2,608,1,41,1,83807.86,1,0,1,112542.58,0
2,3,502,1,42,8,159660.80,3,1,0,113931.57,1
3,4,699,1,39,1,0.00,2,0,0,93826.63,0
4,5,850,1,43,2,125510.82,1,1,1,79084.10,0
5,6,645,0,44,8,113755.78,2,1,0,149756.71,1
6,7,822,0,50,7,0.00,2,1,1,10062.80,0
7,8,376,1,29,4,115046.74,4,1,0,119346.88,1
8,9,501,0,44,4,142051.07,2,0,1,74940.50,0
9,10,684,0,27,2,134603.88,1,1,1,71725.73,0


### Train Test Split 

In [38]:
X = df.drop('Exited', axis='columns')
y = df['Exited']

In [40]:
X.head()

,RowNumber,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,1,619,1,42,2,0.00,1,1,1,101348.88
1,2,608,1,41,1,83807.86,1,0,1,112542.58
2,3,502,1,42,8,159660.80,3,1,0,113931.57
3,4,699,1,39,1,0.00,2,0,0,93826.63
4,5,850,1,43,2,125510.82,1,1,1,79084.10


In [41]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15,
                                                   stratify=y)

In [43]:
y_train.value_counts()

0    5574
1    1426
Name: Exited, dtype: int64

In [44]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [47]:
X_train.shape

(7000, 10)

In [46]:
y_train.shape

(7000,)

In [49]:
X_test.shape

(3000, 10)

In [50]:
y_test.shape

(3000,)

In [56]:
len(X_train.columns)

10

## Build a model in tensorflow (ANN)

In [51]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [66]:
model = keras.Sequential([
    keras.layers.Dense(7, input_dim=10, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [70]:
model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
219/219 [==============================] - 1s 3ms/step - loss: 0.7619 - accuracy: 0.7503
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.7760 - accuracy: 0.7463
Epoch 3/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7293 - accuracy: 0.7504
Epoch 4/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7629 - accuracy: 0.7480
Epoch 5/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7111 - accuracy: 0.7497
Epoch 6/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7180 - accuracy: 0.7524
Epoch 7/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7089 - accuracy: 0.7573
Epoch 8/100
219/219 [==============================] - 1s 2ms/step - loss: 0.6945 - accuracy: 0.7593
Epoch 9/100
219/219 [==============================] - 1s 2ms/step - loss: 0.6961 - accuracy: 0.7567
Epoch 10/100
219/219 [==============================] - 1s 2ms/step - loss: 0.7020 - accura

219/219 [==============================] - 0s 2ms/step - loss: 0.6179 - accuracy: 0.7684
Epoch 82/100
219/219 [==============================] - 1s 2ms/step - loss: 0.6179 - accuracy: 0.7663
Epoch 83/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6161 - accuracy: 0.7726
Epoch 84/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6242 - accuracy: 0.7713
Epoch 85/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6189 - accuracy: 0.7693
Epoch 86/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6144 - accuracy: 0.7731
Epoch 87/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6122 - accuracy: 0.7714
Epoch 88/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6249 - accuracy: 0.7663
Epoch 89/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6123 - accuracy: 0.7741
Epoch 90/100
219/219 [==============================] - 0s 2ms/step - loss: 0.6199 - accuracy: 

In [73]:
model.evaluate(X_test, y_test)

94/94 [==============================] - 1s 2ms/step - loss: 0.6206 - accuracy: 0.7470


[0.6206350326538086, 0.746999979019165]

In [74]:
y_pred = model.predict(X_test)

In [75]:
y_pred[:10]

array([[0.04654232],
       [0.41165322],
       [0.69797826],
       [0.91159725],
       [0.04275388],
       [0.01341268],
       [0.18814394],
       [0.4428277 ],
       [0.13281438],
       [0.11131781]], dtype=float32)

In [77]:
y_test[:10]

9737    0
7380    0
444     1
158     0
1556    0
746     0
4768    1
2504    0
1471    0
4258    0
Name: Exited, dtype: int64

In [78]:
y_predict=[]
for e in y_pred:
    if e > 0.5:
        y_predict.append(1)
    else:
        y_predict.append(0)

In [79]:
y_predict[:10]

[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]

In [80]:
y_test[:10]

9737    0
7380    0
444     1
158     0
1556    0
746     0
4768    1
2504    0
1471    0
4258    0
Name: Exited, dtype: int64

In [81]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      2389
           1       0.38      0.37      0.37       611

    accuracy                           0.75      3000
   macro avg       0.61      0.61      0.61      3000
weighted avg       0.75      0.75      0.75      3000

